In [1]:
from tools.data_reading import CompressedWindowDataset
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader
from tools import utils
from tools.engine import train_one_epoch, evaluate
from model_zoo.InputInjection import InputInjection
from torch.utils.tensorboard import SummaryWriter

In [2]:
dataset = CompressedWindowDataset()
img, spectr, lab = dataset[0]
# the ones with missing end labels are missing (just make code to fill labels!)

torch.Size([2197, 24, 9, 12])
torch.Size([2197, 3, 96, 128])
2197


In [ ]:
for i in range(len(dataset)):
    x = dataset[i]

In [3]:
train = DataLoader(dataset,batch_size = 1, shuffle=True, collate_fn=utils.collate_fn)

In [ ]:
img, spectr, lab = next(iter(train))
img = torch.unsqueeze(img[0], 0).float()
spectr = torch.unsqueeze(spectr[0], 0).float()

In [ ]:
print(img.shape)
print(spectr.shape)
print(lab)

In [4]:
model = InputInjection()
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'running on {device}')
if device == 'cuda':
    device_id = torch.cuda.current_device()
    print(f'using gpu {torch.cuda.get_device_name(device_id)}')
model.to(device)
writer = SummaryWriter()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params,lr=0.00003)

running on cuda


In [5]:
num_epochs = 10
for epoch in range(num_epochs):
    train_one_epoch(writer, model, optimizer, train, device, epoch, print_freq=100)
    evaluate(writer, model, train, device, print_freq=100)
    torch.save(model.state_dict(), f'inputinjection-train-1.weights')

C:\ProgramData\Miniconda3\lib\site-packages\torch\nn\functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


Epoch: [0]  [   0/2197]  eta: 0:44:19  lr: 0.000030  loss: 13.6978 (13.6978)  loss_classifier: 9.8186 (9.8186)  loss_box_reg: 0.4710 (0.4710)  loss_objectness: 1.7654 (1.7654)  loss_rpn_box_reg: 1.6428 (1.6428)  time: 1.2107  data: 0.0010  max mem: 1499
Epoch: [0]  [ 100/2197]  eta: 0:05:56  lr: 0.000030  loss: 2.6498 (5.6000)  loss_classifier: 0.3484 (2.0143)  loss_box_reg: 1.2837 (2.2171)  loss_objectness: 0.0601 (0.2452)  loss_rpn_box_reg: 0.5484 (1.1234)  time: 0.1594  data: 0.0001  max mem: 1614
Epoch: [0]  [ 200/2197]  eta: 0:05:03  lr: 0.000030  loss: 2.8057 (4.5104)  loss_classifier: 0.3906 (1.4733)  loss_box_reg: 0.9850 (1.8958)  loss_objectness: 0.0522 (0.1824)  loss_rpn_box_reg: 0.6726 (0.9589)  time: 0.1759  data: 0.0001  max mem: 1614
Epoch: [0]  [ 300/2197]  eta: 0:04:43  lr: 0.000030  loss: 2.3938 (3.9496)  loss_classifier: 0.4023 (1.2025)  loss_box_reg: 1.0628 (1.6946)  loss_objectness: 0.0509 (0.1548)  loss_rpn_box_reg: 0.5796 (0.8977)  time: 0.1442  data: 0.0000  max 

AttributeError: 'CompressedWindowDataset' object has no attribute 'coco'